In [ ]:
!pip install gym[atari]

In [1]:
%matplotlib widget

In [49]:
from IPython.core.debugger import set_trace

In [2]:
import torch
from torch import nn
from torch.distributions import Categorical
from torch.optim import Adam, SGD

In [3]:
import numpy

In [4]:
from matplotlib import pyplot as plot

In [5]:
import copy
from time import sleep

In [6]:
import gym

In [7]:
device='cpu'

In [8]:
class ResLinear(nn.Module):
    def __init__(self, n_in, n_out, act=nn.ReLU()):
        super(ResLinear, self).__init__()
        self.act = act
        self.linear = nn.Linear(n_in, n_out)
        self.bn = nn.BatchNorm1d(n_out)
        
        assert(n_in == n_out)
    
    def forward(self, x):
        h = self.act(self.bn(self.linear(x)))
        return h + x

In [9]:
class Player(nn.Module):
    def __init__(self, n_in=128, n_hid=100, n_out=6):
        super(Player, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_out))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, obs, normalized=False):
        if normalized:
            return self.softmax(self.layers(obs))
        else:
            return self.layers(obs)

In [59]:
class Qnet(nn.Module):
    def __init__(self, n_in=128, n_act=6, n_hid=100):
        super(Qnet, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid), 
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_act))
    
    def forward(self, obs, act):
#         set_trace()
        return self.layers(obs).gather(1, act.long())
    
    def value(self, obs):
        return self.layers(obs).max(dim=1)
    
    def q(self, obs):
        return self.layers(obs)

In [24]:
class Value(nn.Module):
    def __init__(self, n_in=128, n_hid=100):
        super(Value, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, 1))
    
    def forward(self, obs):
        return self.layers(obs)

In [25]:
def copy_params(from_, to_):
    for f_, t_ in zip(from_.parameters(), to_.parameters()):
        t_.data.copy_(f_.data)
        
def avg_params(from_, to_, coeff=0.95):
    for f_, t_ in zip(from_.parameters(), to_.parameters()):
        t_.data.copy_(coeff * t_.data + (1.-coeff) * f_.data)

In [26]:
def normalize_obs(obs):
    return obs.astype('float32') / 255.

In [27]:
# collect data
def collect_one_episode(env, player, max_len=50, discount_factor=0.9, deterministic=False, rendering=False, verbose=False):
    episode = []

    observations = []

    rewards = []
    crewards = []

    actions = []
    action_probs = []

    obs = env.reset()
    
    for ml in range(max_len):
        if rendering:
            env.render()
            sleep(0.05)
            
        obs = normalize_obs(obs)

        out_probs = player(torch.from_numpy(obs[None,:]).to(device), normalized=True).squeeze()
        
        if deterministic:
            action = numpy.argmax(out_probs.to('cpu').data.numpy())
            if verbose:
                print(out_probs, action)
        else:
            act_dist = Categorical(out_probs)
            action = act_dist.sample().item()
        action_prob = out_probs[action].item()

        observations.append(obs)
        actions.append(action)
        action_probs.append(action_prob)

        obs, reward, done, info = env.step(action)
        if deterministic and verbose:
            print(reward, done)
        
        rewards.append(reward)

    rewards = numpy.array(rewards)

    # it's probably not the best idea to compute the discounted cumulative returns here, but well..
    for ri in range(len(rewards)):
        factors = (discount_factor ** numpy.arange(len(rewards)-ri))
        crewards.append(numpy.sum(rewards[ri:] * factors))
        
    # discard the final 10%, because it really doesn't give me a good signal due to the unbounded horizon
    # this is only for training, not for computing the total return of the episode of the given length
    discard = max_len // 10
        
    return observations[:-discard], rewards[:-discard], crewards[:-discard], actions[:-discard], action_probs[:-discard], rewards.sum()

In [35]:
# simple implementation of FIFO-based replay buffer
class Buffer:
    def __init__(self, max_items=10000):
        self.max_items = max_items
        self.buffer = []
        
    def add(self, observations, rewards, crewards, actions, action_probs):
        new_n = len(observations)
        old_n = len(self.buffer)
        if new_n + old_n > self.max_items:
            del self.buffer[:new_n]
        for o, r, c, a, p, on, rn, cn, an, pn in zip(observations[:-1], rewards[:-1], crewards[:-1], actions[:-1], action_probs[:-1],
                                             observations[1:], rewards[1:], crewards[1:], actions[1:], action_probs[1:]):
            self.buffer.append({'current': {'obs': o, 
                                            'rew': r, 
                                            'crew': c, 
                                            'act': a, 
                                            'prob': p},
                                'next': {'obs': on, 
                                         'rew': rn, 
                                         'crew': cn, 
                                         'act': an, 
                                         'prob': pn}})
            
    def sample(self, n=100):
        idxs = numpy.random.choice(len(self.buffer),n)
        return [self.buffer[ii] for ii in idxs]

In [30]:
env = gym.make('Pong-ram-v0')

In [77]:
# create a policy
player = Player(n_in=128, n_hid=128, n_out=6).to(device)

In [78]:
# create a q estimator
qnet = Qnet(n_in=128, n_hid=128, n_act=6).to(device)
qold = Qnet(n_in=128, n_hid=128, n_act=6).to(device)
copy_params(qnet, qold)

In [79]:
# create a value estimator
value = Value(n_in=128, n_hid=128).to(device)

In [80]:
# initialize optimizers
opt_player = Adam(player.parameters(), lr=0.0001)
opt_q = Adam(qnet.parameters(), lr=0.0001)
opt_value = Adam(value.parameters(), lr=0.001)

In [81]:
# initialize replay buffer
replay_buffer = Buffer(max_items=50000)

In [ ]:
n_iter = 1000
init_collect = 1
n_collect = 1
n_q = 50
n_value = 50
n_policy = 50
disp_iter = 1
val_iter = 1

max_len = 1000
batch_size = 1000

ent_coeff = 0. #0.001
discount_factor = .95

q_loss = -numpy.Inf
value_loss = -numpy.Inf
ret = -numpy.Inf
entropy = -numpy.Inf
valid_ret = -numpy.Inf

return_history = []

for ni in range(n_iter):
    player.eval()

    if numpy.mod(ni, val_iter) == 0:
        _, _, _, _, _, ret_ = collect_one_episode(env, player, max_len=max_len, deterministic=True)
        return_history.append(ret_)
        if valid_ret == -numpy.Inf:
            valid_ret = ret_
        else:
            valid_ret = 0.9 * valid_ret + 0.1 * ret_
        print('Valid run', ret_, valid_ret)

    # collect some episodes using the current policy
    # and push (obs,a,r,p(a)) tuples to the replay buffer.
    nc = n_collect
    if ni == 0:
        nc = init_collect
    for ci in range(nc):
        o_, r_, c_, a_, ap_, ret_ = collect_one_episode(env, player, max_len=max_len, discount_factor=discount_factor)
        replay_buffer.add(o_, r_, c_, a_, ap_)
        if ret == -numpy.Inf:
            ret = ret_
        else:
            ret = 0.9 * ret + 0.1 * ret_
    
    player.train()
    
    # fit a q function
    for qi in range(n_q):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_xn = torch.from_numpy(numpy.stack([ex['next']['obs'] for ex in batch]).astype('float32')).to(device)
        
        batch_y = torch.from_numpy(numpy.stack([ex['current']['rew'] for ex in batch]).astype('float32')).to(device)
        
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        
        q_pred = qnet(batch_x, batch_a).squeeze()
        q_next = qold.value(batch_xn)[0].squeeze()
        
        loss_ = ((q_pred - (batch_y + discount_factor * q_pred)) ** 2)
        
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        batch_pi = player(batch_x, normalized=True)
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_q.step()
        
    if q_loss == -numpy.Inf:
        q_loss = loss_.mean().item()
    else:
        q_loss = 0.9 * q_loss + 0.1 * loss_.mean().item()
        
    copy_params(qnet, qold)
        
    # fit a value function
    for vi in range(n_value):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        
        pred_y = value(batch_x).squeeze()
        pred_q = qnet(batch_x, batch_a)[0].squeeze()
        
        loss_ = ((pred_y - pred_q) ** 2)
        
        batch_pi = player(batch_x, normalized=True)
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_value.step()
        
    if value_loss < 0.:
        value_loss = loss_.mean().item()
    else:
        value_loss = 0.9 * value_loss + 0.1 * loss_.mean().item()
    
    if numpy.mod(ni, disp_iter) == 0:
        print('# plays', (ni+1) * n_collect, 'return', ret, 'value_loss', value_loss, 'q_loss', q_loss, 'entropy', -entropy)
    
    # fit a policy
    for pi in range(n_policy):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)

#         batch_v = value(batch_x)
        batch_v = qnet.value(batch_x)[0]
        batch_q = qnet(batch_x, batch_a)
        
        batch_pi = player(batch_x, normalized=True)
        
        logp = torch.log(batch_pi.gather(1, batch_a.long()))
        
        # advantage
        adv = batch_q.clone().detach() - batch_v.clone().detach()
        
        loss = -(adv * logp)
        
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss
        
        # entropy regularization: though, it doesn't look necessary in this specific case.
        ent = (batch_pi * torch.log(batch_pi)).sum(1)
        
        if entropy == -numpy.Inf:
            entropy = ent.mean().item()
        else:
            entropy = 0.9 * entropy + 0.1 * ent.mean().item()
        
        loss = (loss + ent_coeff * ent).mean()
        
        loss.backward()
        opt_player.step()

Valid run -20.0 -20.0
# plays 1 return -16.0 value_loss 0.02605981007218361 q_loss 0.013791528530418873 entropy inf
Valid run -15.0 -19.5
# plays 2 return -16.3 value_loss 0.024653300270438192 q_loss 0.013632574398070575 entropy 1.7467347790864332
Valid run -9.0 -18.45
# plays 3 return -16.270000000000003 value_loss 0.033247423581779 q_loss 0.013662318764254453 entropy 1.7430345535599252
Valid run -20.0 -18.605
# plays 4 return -16.243000000000002 value_loss 0.03648584626242518 q_loss 0.013870261016301812 entropy 1.7341013453573104
Valid run -20.0 -18.744500000000002
# plays 5 return -16.618700000000004 value_loss 0.03465242442153394 q_loss 0.014718127461802216 entropy 1.7230722022331884
Valid run -20.0 -18.870050000000003
# plays 6 return -16.956830000000004 value_loss 0.03329117602106184 q_loss 0.015449385460494087 entropy 1.7092774051620379
Valid run -20.0 -18.983045000000004
# plays 7 return -16.861147000000003 value_loss 0.03009503423940763 q_loss 0.015261248025695794 entropy 1.69

In [21]:
plot.figure()

plot.plot(return_history)
plot.grid(True)
plot.xlabel('# of plays x {}'.format(n_collect))
plot.ylabel('Return over the episode of length {}'.format(max_len))

plot.show()
plot.savefig('return_log.pdf', dpi=150)

FigureCanvasNbAgg()

In [22]:
# let the final policy play the pong longer
player.eval()
_, _, _, _, ret_ = collect_one_episode(env, player, max_len=1000000, deterministic=True, rendering=True)

KeyboardInterrupt: 